In [1]:


from sklearn.feature_extraction.text import TfidfVectorizer
# تحميل البيانات المعالجة في أي نوت بوك تاني
import pandas as pd
data = pd.read_csv("processed_movielens_100k.csv")

In [2]:
import pandas as pd
from sklearn.decomposition import TruncatedSVD

# Pivot table: rows = users, columns = movies, values = ratings
ratings_matrix = data.pivot_table(index='userId', columns='movieId', values='rating').fillna(0)

In [3]:
# تطبيق SVD - نختار عدد المكونات (الميزات الكامنة)
svd = TruncatedSVD(n_components=20, random_state=42)
latent_matrix = svd.fit_transform(ratings_matrix)


In [4]:
reconstructed_matrix = svd.inverse_transform(latent_matrix)
predicted_ratings = pd.DataFrame(reconstructed_matrix, index=ratings_matrix.index, columns=ratings_matrix.columns)


In [5]:
def recommend_by_user_svd(user_id, predicted_ratings, data, top_n=10):
    # ترتيب الأفلام حسب التوقعات
    user_ratings = predicted_ratings.loc[user_id]
    
    # شيل الأفلام اللي المستخدم شافها فعلًا
    watched_movies = data[data['userId'] == user_id]['movieId'].tolist()
    user_ratings = user_ratings.drop(labels=watched_movies)
    
    # ترتيب التوصيات
    top_movies = user_ratings.sort_values(ascending=False).head(top_n).index
    return data[data['movieId'].isin(top_movies)][['title']].drop_duplicates().head(top_n)


In [9]:
recommend_by_user_svd(user_id=10, predicted_ratings=predicted_ratings, data=data, top_n=5)


,title
231,"Shawshank Redemption, The (1994)"
245,Inception (2010)
469,Harry Potter and the Sorcerer's Stone (a.k.a. ...
891,"Monsters, Inc. (2001)"
909,Pirates of the Caribbean: The Curse of the Bla...
